In [2]:
import numpy as np
import pandas as pd
import building_blocks as lrb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

# Let's start by looking at an example with 3 teams.

In [3]:
# Set seed for reproducibility
np.random.seed(42)

# --- Step 1: Define teams and latent strengths ---
teams = ['A', 'B', 'C']
team_ids = {name: i for i, name in enumerate(teams)}
true_betas = np.array([0.0, 0.05, -0.05])  # beta_A=0, beta_B=0.05, beta_C=-0.05


In [4]:
# --- Step 2: Simulate 20 random matchups ---
n_games = 20
matchups = []

for _ in range(n_games):
    i, j = np.random.choice(3, size=2, replace=False) # randomly choose 2 teams to compete.
    beta_diff = true_betas[i] - true_betas[j]
    prob_win_i = 1 / (1 + np.exp(-beta_diff))
    winner = i if np.random.rand() < prob_win_i else j
    matchups.append((i, j, winner))

In [6]:
# --- Step 3: Create design matrix and outcome vector ---
X = np.zeros((n_games, 2))  # We fix beta_0 = 0 and estimate beta_1 and beta_2.
y = np.zeros(n_games)

for idx, (i, j, winner) in enumerate(matchups):
    # Map to reduced index space (beta_0 = 0)
    def reduced(k): return k - 1 if k > 0 else None
    
    if winner == i:
        y[idx] = 1
        if reduced(i) is not None:
            X[idx, reduced(i)] += 1
        if reduced(j) is not None:
            X[idx, reduced(j)] -= 1
    else:
        y[idx] = 0
        if reduced(j) is not None:
            X[idx, reduced(j)] += 1
        if reduced(i) is not None:
            X[idx, reduced(i)] -= 1

In [7]:
# --- Step 4: Display the data ---
match_df = pd.DataFrame(matchups, columns=["team_i", "team_j", "winner"])
match_df["team_i"] = match_df["team_i"].map({v: k for k, v in team_ids.items()})
match_df["team_j"] = match_df["team_j"].map({v: k for k, v in team_ids.items()})
match_df["winner"] = match_df["winner"].map({v: k for k, v in team_ids.items()})
print("Match Outcomes:")
print(match_df)

print("\nDesign Matrix X:")
print(X)

print("\nOutcome Vector y:")
print(y)

Match Outcomes:
   team_i team_j winner
0       A      B      B
1       A      B      B
2       A      B      A
3       B      A      A
4       A      B      A
5       A      C      A
6       B      C      B
7       C      A      C
8       B      C      C
9       A      C      A
10      A      B      A
11      C      A      C
12      B      C      C
13      B      A      B
14      A      C      C
15      A      C      A
16      C      A      C
17      A      B      A
18      C      B      C
19      C      A      C

Design Matrix X:
[[ 1.  0.]
 [ 1.  0.]
 [-1.  0.]
 [-1.  0.]
 [-1.  0.]
 [ 0. -1.]
 [ 1. -1.]
 [ 0.  1.]
 [-1.  1.]
 [ 0. -1.]
 [-1.  0.]
 [ 0.  1.]
 [-1.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0. -1.]
 [ 0.  1.]
 [-1.  0.]
 [-1.  1.]
 [ 0.  1.]]

Outcome Vector y:
[0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1.]


## Now, let's see if these rankings are robust to dropping just a little data.

### Fit a logistic regression model to the full data.

In [8]:
full_model = lrb.run_logistic_regression(X, y)
full_model

LogisticRegression(fit_intercept=False, penalty=None)

In [9]:
pos_p_hats = full_model.predict_proba(X)[:, 1]
pos_p_hats

array([0.37499992, 0.37499992, 0.62500008, 0.62500008, 0.62500008,
       0.62500008, 0.5       , 0.37499992, 0.5       , 0.62500008,
       0.62500008, 0.37499992, 0.5       , 0.37499992, 0.37499992,
       0.62500008, 0.37499992, 0.62500008, 0.5       , 0.37499992])

In [10]:
e_A = np.array([1, 0]).reshape(1, 2) # for beta_A > 0
e_A.shape
e_AminusB = np.array([1, -1]).reshape(1, 2) # for beta_A > beta_B
e_AminusB.shape

(1, 2)

In [16]:
index = 1
v_lst = pos_p_hats * (1 - pos_p_hats)
V = np.diag(v_lst)
# a vector of size num dimension (e.g., num teams - 1)
np.linalg.inv(X.T @ V @ X) @ X[index] * (y[index] - pos_p_hats[index])

array([[-0.14838708, -0.0516129 ],
       [-0.0516129 , -0.14838708]])

### Approximately how many points need to be dropped for $\beta_A > 0$ to no longer hold?

In [11]:
# how many data points must be dropped to change beta_A.
scores = []
# compute the influence scores for all points.
for ind in range(1, len(X)):
    score_i = lrb.compute_approx(pos_p_hats, X, ind, y, "1sN")
    scores.append(score_i)
scores

[-0.16355553214578855,
 -0.1635555321457885,
 0.2725926461687005,
 -0.1635555321457885,
 -0.056888883896953325,
 0.1481481551373098,
 0.09481483870097142,
 0.1481481551373098,
 -0.056888883896953325,
 -0.1635555321457885,
 0.09481483870097142,
 0.1481481551373098,
 0.2725926461687005,
 -0.056888883896953325,
 -0.056888883896953325,
 0.09481483870097142,
 -0.1635555321457885,
 -0.1481481551373098,
 0.09481483870097142]

In [12]:
# sort indices in ascending order.
inds = np.argsort(scores)
# MIS. identify the points with the most negative (direction chosen arbitrarily) scores.
print(f'the games with the most negative scores are: {inds[-10:]}')

the games with the most negative scores are: [14  6 10 15 18  5  7 11 12  2]


In [13]:
scores_array = np.array(scores)
scores_array[inds]

array([-0.16355553, -0.16355553, -0.16355553, -0.16355553, -0.16355553,
       -0.14814816, -0.05688888, -0.05688888, -0.05688888, -0.05688888,
        0.09481484,  0.09481484,  0.09481484,  0.09481484,  0.14814816,
        0.14814816,  0.14814816,  0.27259265,  0.27259265])

### Approximately how many points need to be dropped for $\beta_A > \beta_B$ to no longer hold?

### Helpers.. (for now).

In [31]:
def compute_leverage(
    pos_p_hats: np.ndarray,
    X: np.ndarray,
    index: int,
    y: np.ndarray,
) -> float:
    """
    pos_p_hats: np.array, shape (n,), the predicted probabilities.
    X: np.array, shape (n, p), the design matrix.
    index: int, the index of the data point whose influence we want to compute.
    y: np.array, shape (n,), the response variable.

    Compute the leverage of the index-th data point.
    """
    v_lst = pos_p_hats * (1 - pos_p_hats)
    V = np.diag(v_lst)
    H = V @ X @ np.linalg.inv(X.T @ V @ X) @ X.T
    return H[index, index]

def compute_approx(
    pos_p_hats: np.ndarray,
    X: np.ndarray,
    index: int,
    y: np.ndarray,
    method: str,
    e: np.ndarray,
) -> float:
    """
    pos_p_hats: np.array, shape (n,), the predicted probabilities.
    X: np.array, shape (n, p), the design matrix.
    index: int, the index of the data point whose influence we want to compute.
    y: np.array, shape (n,), the response variable.
    method: str, the method to use to compute the approximation ("1sN" or "IF").
    e: np.ndarray, shape (p,), the direction of the influence function.

    Compute the influence function approximation of
    the effect of infinitesimally upweighting the index-th
    data point on a quantity of interest 
    (e.g., some linear combination of the 
    logistic regression coefficients,
    determined by the choice of e).
    """
    v_lst = pos_p_hats * (1 - pos_p_hats)
    V = np.diag(v_lst)
    if method == "IF":
        influence_function = (
            e @ np.linalg.inv(X.T @ V @ X) @ X[index] * (y[index] - pos_p_hats[index])
        )
        return influence_function[0]
    elif method == "1sN":
        influence_function = (
            e @ np.linalg.inv(X.T @ V @ X) @ X[index] * (y[index] - pos_p_hats[index])
        )
        h_ii = compute_leverage(pos_p_hats, X, index, y)
        return 1 / (1 - h_ii) * influence_function[0]
    else:
        return "Invalid method."